In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"


In [2]:
import logging
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [3]:
from models.data import ArabicAbstractsDataModule

In [4]:
# Initialize the data module
# data_module = HC3TextDataModule()
data_module = ArabicAbstractsDataModule()

data_module.setup()

In [5]:
# Define the model (you can switch between different models)

from models.models import LitXLMRobertaModel
import pytorch_lightning as pl
from pytorch_lightning.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
)


In [6]:
class CrossPromptExperiment:
    def __init__(self, max_epochs=100):
        self.max_epochs = max_epochs
        self.results = {}
        self.checkpoints_path = "trained_detectors/Arabic/ArabicAbstractsDataset/{train_prompt}AIDetector/checkpoints"
        
        # Map the long prompt names to shorter versions for file paths
        self.prompt_map = {
            "by_polishing_abstracts_abstracts_generation_filtered": "polishing",
            "from_title_abstracts_generation_filtered": "title_only",
            "from_title_and_content_abstracts_generation_filtered": "title_content"
        }
        
    def _get_callbacks(self, train_prompt):
        early_stopping = EarlyStopping(
            monitor="val_loss",
            min_delta=0.0,
            patience=3,
            verbose=True,
            mode="min",
        )
        
        # Use shortened prompt name for checkpoint path
        short_prompt = self.prompt_map[train_prompt]
        checkpoint = ModelCheckpoint(
            monitor="val_loss",
            dirpath=self.checkpoints_path.format(train_prompt=short_prompt.title()),
            filename="best-checkpoint",
            save_top_k=1,
            mode="min",
        )
        
        return [early_stopping, checkpoint]
    
    def _test_on_prompt(self, trainer, model, test_prompt, train_prompt):
        # Load the best checkpoint before testing
        short_prompt = self.prompt_map[train_prompt]
        checkpoint_path = self.checkpoints_path.format(train_prompt=short_prompt.title())
        checkpoint_path += '/best-checkpoint.ckpt'
        model = LitXLMRobertaModel.load_from_checkpoint(checkpoint_path)
        
        # Create test datamodule with all models but only the test prompt
        test_datamodule = ArabicAbstractsDataModule(
            models=["allam", "jais-batched", "llama-batched"],
            generation_types=[test_prompt]
        )
        test_datamodule.setup()
        
        results = trainer.test(model, test_datamodule.test_dataloader())[0]
        return {
            'accuracy': results['test_acc'],
            'precision': results['test_precision'],
            'recall': results['test_recall'],
            'f1': results['test_f1'],
            'loss': results['test_loss']
        }
    
    def run_experiment(self, train_prompt, test_prompts):
        # Initialize components
        model = LitXLMRobertaModel()
        
        # Create training datamodule with all models but only the training prompt
        train_datamodule = ArabicAbstractsDataModule(
            models=["allam", "jais-batched", "llama-batched"],
            generation_types=[train_prompt]
        )
        
        trainer = pl.Trainer(
            devices=1,
            max_epochs=self.max_epochs,
            accelerator="auto",
            val_check_interval=0.25,
            check_val_every_n_epoch=1,
            callbacks=self._get_callbacks(train_prompt),
        )
        
        # Train the model
        print(f"\nTraining on {self.prompt_map[train_prompt]} prompt data...")
        trainer.fit(model, train_datamodule)
        
        # Test on all specified prompts
        results = {}
        for test_prompt in test_prompts:
            print(f"\nTesting on {self.prompt_map[test_prompt]} prompt data...")
            results[test_prompt] = self._test_on_prompt(trainer, model, test_prompt, train_prompt)
        
        # Store results
        self.results[train_prompt] = results
        
        # Display results
        self._display_results(train_prompt, results)
        
        # Print checkpoint location
        short_prompt = self.prompt_map[train_prompt]
        checkpoint_dir = self.checkpoints_path.format(train_prompt=short_prompt.title())
        print(f"\nBest model checkpoint saved at: {checkpoint_dir}/best-checkpoint.ckpt")
        
        return results
    
    def _display_results(self, train_prompt, results):
        print(f"\nResults for model trained on {self.prompt_map[train_prompt]} prompt:")
        print("-" * 80)
        print(f"{'Test Prompt':<20} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1':<10} {'Loss':<10}")
        print("-" * 80)
        
        for test_prompt, metrics in results.items():
            print(
                f"{self.prompt_map[test_prompt]:<20}"
                f"{metrics['accuracy']:<10.4f}"
                f"{metrics['precision']:<10.4f}"
                f"{metrics['recall']:<10.4f}"
                f"{metrics['f1']:<10.4f}"
                f"{metrics['loss']:<10.4f}"
            )
        print("-" * 80)

In [8]:
# Get list of available prompts
available_prompts = ArabicAbstractsDataModule.GENERATION_TYPES

# Initialize and run the cross-prompt experiment
experiment = CrossPromptExperiment()

all_results = {}
for train_prompt in available_prompts:
    print(f"\n{'='*50}")
    print(f"Training on {experiment.prompt_map[train_prompt]} prompt")
    print(f"{'='*50}")
    
    results = experiment.run_experiment(
        train_prompt=train_prompt,
        test_prompts=available_prompts
    )
    all_results[train_prompt] = results


Training on polishing prompt


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[e


Training on polishing prompt data...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name            | Type                                | Params | Mode 
---------------------------------------------------------------------------------
0  | val_accuracy    | BinaryAccuracy                      | 0      | train
1  | test_accuracy   | BinaryAccuracy                      | 0      | train
2  | train_accuracy  | BinaryAccuracy                      | 0      | train
3  | xlm_roberta     | XLMRobertaForSequenceClassification | 278 M  | eval 
4  | fc              | Linear                              | 3      | train
5  | activation      | Sigmoid                             | 0      | train
6  | train_precision | BinaryPrecision                     | 0      | train
7  | val_precision   | BinaryPrecision                     | 0      | train
8  | test_precision  | BinaryPrecision                     | 0      | train
9  | train_recall    | BinaryRecall                        | 0      | train
10 | val_recall      | BinaryRecall    

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...


Testing on title_content prompt data...


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9889967441558838     │
│          test_f1          │    0.9926022291183472     │
│         test_loss         │    0.03038187325000763    │
│      test_precision       │    0.9898106455802917     │
│        test_recall        │     0.995736300945282     │
└───────────────────────────┴───────────────────────────┘


Results for model trained on title_content prompt:
--------------------------------------------------------------------------------
Test Prompt          Accuracy   Precision  Recall     F1         Loss      
--------------------------------------------------------------------------------
polishing           0.9871    0.9966    0.9863    0.9913    0.0523    
title_only          0.9949    0.9958    0.9979    0.9968    0.0159    
title_content       0.9890    0.9898    0.9957    0.9926    0.0304    
--------------------------------------------------------------------------------

Best model checkpoint saved at: outputs/Arabic/ArabicAbstractsDataset/Title_ContentAIDetector/checkpoints/best-checkpoint.ckpt


In [ ]:
exit()

: 